In [1]:
from ecnet.model import load_model

cn_model = load_model('_models/cn.pt')
ysi_model = load_model('_models/ysi.pt')
kv_model = load_model('_models/kv.pt')
cp_model = load_model('_models/cp.pt')
lhv_model = load_model('_models/lhv.pt')

with open('_models/cn_desc.txt', 'r') as txt_file:
    desc_cn = txt_file.readlines()
txt_file.close()
desc_cn = [int(i) for i in desc_cn]
with open('_models/ysi_desc.txt', 'r') as txt_file:
    desc_ysi = txt_file.readlines()
txt_file.close()
desc_ysi = [int(i) for i in desc_ysi]
with open('_models/kv_desc.txt', 'r') as txt_file:
    desc_kv = txt_file.readlines()
txt_file.close()
desc_kv = [int(i) for i in desc_kv]
with open('_models/cp_desc.txt', 'r') as txt_file:
    desc_cp = txt_file.readlines()
txt_file.close()
desc_cp = [int(i) for i in desc_cp]
with open('_models/lhv_desc.txt', 'r') as txt_file:
    desc_lhv = txt_file.readlines()
txt_file.close()
desc_lhv = [int(i) for i in desc_lhv]

In [2]:
from csv import DictReader
from ecnet.datasets import QSPRDataset
from copy import deepcopy

with open('_data/fp_cu_data.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    compounds = [r for r in reader]
csv_file.close()
smiles = [c['SMILES'] for c in compounds]

dataset = QSPRDataset(smiles=smiles, target_vals=[0 for _ in range(len(smiles))], backend='alvadesc')

In [3]:
dataset_cn = deepcopy(dataset)
dataset_cn.set_desc_index(desc_cn)
dataset_ysi = deepcopy(dataset)
dataset_ysi.set_desc_index(desc_ysi)
dataset_kv = deepcopy(dataset)
dataset_kv.set_desc_index(desc_kv)
dataset_cp = deepcopy(dataset)
dataset_cp.set_desc_index(desc_cp)
dataset_lhv = deepcopy(dataset)
dataset_lhv.set_desc_index(desc_lhv)

In [4]:
cn_pred = cn_model(dataset_cn.desc_vals).detach().numpy()
ysi_pred = ysi_model(dataset_ysi.desc_vals).detach().numpy()
kv_pred = kv_model(dataset_kv.desc_vals).detach().numpy()
cp_pred = cp_model(dataset_cp.desc_vals).detach().numpy()
lhv_pred = lhv_model(dataset_lhv.desc_vals).detach().numpy()

In [5]:
from csv import DictWriter
with open('_results/fp_cu_results.csv', 'w', encoding='utf8') as csv_file:
    writer = DictWriter(csv_file, ['Name', 'SMILES', 'CN', 'YSI', 'KV', 'CP', 'LHV'],
                        delimiter=',', lineterminator='\n')
    writer.writeheader()
    for i in range(len(cn_pred)):
        writer.writerow({
            'Name': compounds[i]['Name'],
            'SMILES': smiles[i],
            'CN': cn_pred[i][0],
            'YSI': ysi_pred[i][0],
            'KV': kv_pred[i][0],
            'CP': cp_pred[i][0],
            'LHV': lhv_pred[i][0]
        })
csv_file.close()